In [1]:
!pip install xformers
!pip install transformers[torch]
!pip install datasets
!pip install  torchtext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from transformers import set_seed,  GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, GPT2LMHeadModel, pipeline, Trainer, TrainingArguments
from torch import tensor, numel
import pandas as pd
from datasets import Dataset

set_seed(42)

In [52]:
df = pd.read_csv("/content/drive/MyDrive/Chatbot/Data/Data-For-GPT2.csv")

In [53]:
df.head()

,Question,Answer
0,What's your favorite color?,"I'm just a chatbot, so I don't have a favorite..."
1,Tell me a joke.,Why did the chicken cross the road? To get to ...
2,How's the weather today?,"I'm sorry, I don't have access to real-time we..."
3,What's the meaning of life?,The meaning of life is a philosophical questio...
4,What do you like to do in your free time?,"I don't have free time, but I'm here to help you!"


In [54]:

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token



In [55]:
CONVERSION_PROMPT = "Q&A\n"
CONVERSION_TOKEN = "Answer: "


In [56]:
training_examples = f'{CONVERSION_PROMPT}Question: ' + df['Question'] + '\n' + CONVERSION_TOKEN + ' ' + df['Answer'].astype(str)

print(training_examples[10])

Q&A
Question: Tell me something interesting.
Answer:  Sure! Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible.


In [57]:
training_examples

0      Q&A\nQuestion: What's your favorite color?\nAn...
1      Q&A\nQuestion: Tell me a joke.\nAnswer:  Why d...
2      Q&A\nQuestion: How's the weather today?\nAnswe...
3      Q&A\nQuestion: What's the meaning of life?\nAn...
4      Q&A\nQuestion: What do you like to do in your ...
                             ...                        
173    Q&A\nQuestion: I'm feeling stressed about an u...
174    Q&A\nQuestion: Can you share a funny anecdote ...
175    Q&A\nQuestion: I could use some positivity. Sh...
176    Q&A\nQuestion: I'm feeling stressed out. How d...
177    Q&A\nQuestion: Tell me a silly joke that'll ma...
Length: 178, dtype: object

In [58]:
task_df = pd.DataFrame({'text': training_examples[:1200]})

task_df.head(2)

,text
0,Q&A\nQuestion: What's your favorite color?\nAn...
1,Q&A\nQuestion: Tell me a joke.\nAnswer: Why d...


In [28]:
latex_data = Dataset.from_pandas(task_df)

In [29]:
latex_data

Dataset({
    features: ['text'],
    num_rows: 1200
})

In [30]:
def preprocess(examples):  # tokenize our text but don't pad because our collator will pad for us dynamically
    return tokenizer(examples['text'], truncation=True)

latex_data = latex_data.map(preprocess, batched=True)

latex_data = latex_data.train_test_split(train_size=.8)

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [31]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [32]:
model_gpt = GPT2LMHeadModel.from_pretrained('gpt2')


In [33]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Chatbot/Output_GPT_2_3",
        overwrite_output_dir=True,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=50,
        save_steps=10_000,
        save_total_limit=2,

)


    # output_dir="/content/drive/MyDrive/Chatbot/Output_GPT_2",
    # overwrite_output_dir=True, #overwrite the content of the output directory
    # num_train_epochs=10, # number of training epochs
    # per_device_train_batch_size=2, # batch size for training
    # per_device_eval_batch_size=20,  # batch size for evaluation
    # load_best_model_at_end=True,
    # logging_steps=40,
    # log_level='info',
    # # evaluation_strategy='no',
    # # save_strategy='no',
    # save_total_limit = 1,



trainer = Trainer(
    model=model_gpt,
    args=training_args,
    train_dataset=latex_data["train"],
    eval_dataset=latex_data["test"],
    data_collator=data_collator,
)

trainer.evaluate()

{'eval_loss': 4.311465263366699,
 'eval_runtime': 1.1353,
 'eval_samples_per_second': 211.393,
 'eval_steps_per_second': 52.848}

In [34]:
trainer.train()

Step,Training Loss
500,1.502700
1000,0.903900
1500,0.661600
2000,0.524800
2500,0.439900
3000,0.385100
3500,0.357700
4000,0.339200
4500,0.328600
5000,0.318700


TrainOutput(global_step=12000, training_loss=0.41066673151652017, metrics={'train_runtime': 1247.5253, 'train_samples_per_second': 38.476, 'train_steps_per_second': 9.619, 'total_flos': 799094315520000.0, 'train_loss': 0.41066673151652017, 'epoch': 50.0})

In [35]:
trainer.evaluate()

{'eval_loss': 1.6014885902404785,
 'eval_runtime': 1.1845,
 'eval_samples_per_second': 202.612,
 'eval_steps_per_second': 50.653,
 'epoch': 50.0}

In [36]:
trainer.save_model()

In [45]:
loaded_model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/Chatbot/Output_GPT_2_2')
generator = pipeline('text-generation', model=loaded_model, tokenizer=tokenizer)

In [51]:
texts = ['What makes you sad?',
         "go on comrade",
        "who is Batman?",
         "what happened to my happines?",
         "Do a chatbot like you likes chocolates?",
         "Apple or microsoft?"
         ]


data = []

for t in texts:

    sample_= f'{CONVERSION_PROMPT}Question: {t}\n{CONVERSION_TOKEN}'

    data.append(sample_)




for i in data:



    res = generator(
        i, temperature=0.1,
        max_length=len(tokenizer.encode(i)) + 35
    )[0]['generated_text']

    print()
    print()
    print(res)
    print()
    print()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




Q&A
Question: What makes you sad?
Answer:  Sad is a subjective experience, and its keys can vary from person to person. It can be a source of joy and stress. Keeping it a secret can help. It can




Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




Q&A
Question: go on comrade
Answer:  I'm feeling stressed about an upcoming exam. How can I stay calm and focused?
Answer:  Take breaks, practice deep breathing, and maintain a study schedule. You




Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




Q&A
Question: who is Batman?
Answer:  Batman is a mythical creature, with a powerful and powerful connection to the human resulting in a strong and loving relationship. He is often associated with dark magic, often associated with dark




Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




Q&A
Question: what happened to my happines?
Answer:  I don't have happines, but I'm here to assist with your questions and tasks. I'm here to assist with tasks and provide information. What do you call a




Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




Q&A
Question: Do a chatbot like you likes chocolates?
Answer:  I don't eat chocolate, but I can provide recipes and cooking tips. What about you? Tell me, what do you like to do in your free time?
Answer




Q&A
Question: Apple or microsoft?
Answer:  I don't have personal preferences, but I can recommend products based on your interests. What are some key principles of computer and information security?
Answer:  Deciding to


